In [1]:
import os
os.listdir("results")

['prelim', 'ESD_pseudo_reality', 'Emulator_hist_future']

In [2]:
import os

def check_model_availability():
    # Configuration options
    domains = ["ALPS", "NZ", "SA"]
    training_experiments = ["Emulator_hist_future", "ESD_pseudo_reality"]
    add_orography_options = [True, False]
    
    # Model configuration definitions from your logic
    configs = [
        {"norm": "pw", "var": "tasmax", "alpha": False},
        {"norm": "subtract_linear", "var": "tasmax", "alpha": True},
        {"norm": "scale_pw", "var": "pr", "alpha": False},
        {"norm": "subtract_linear", "var": "pr", "alpha": True}
    ]

    missing_models = []
    found_count = 0
    total_count = 0

    print(f"{'STATUS':<10} | {'PATH'}")
    print("-" * 80)

    for exp in training_experiments:
        for dom in domains:
            for orog in add_orography_options:
                for conf in configs:
                    if conf["norm"] == "subtract_linear" and orog:
                        # Skip this configuration if orography is enabled and norm is subtract_linear
                        continue
                    # Logic to determine model_dirs and burn_ins based on your snippet
                    norm = conf["norm"]
                    var = conf["var"]
                    
                    # Handle alpha string logic
                    # Assuming default alpha is 100 as per your snippet
                    if dom == "ALPS":
                        alpha_val = 100 
                        alpha_str = "" # Default
                    else:
                        alpha_val = 1000
                        alpha_str = f"_alpha-{alpha_val}" if conf["alpha"] else ""
                    
                    # Reconstructing the model_dirs list for each config
                    if norm == "pw" and var == "tasmax":
                        model_dirs = [
                            "coarse/var-tasmax/hd-100_num-lay-6_norm-out-normalise_pw_norm-in-normalise_scalar_nd-100_sorted-pred/",
                            "super/lr16_hr8/var-tasmax/loc-specific-layers_norm-out-normalise_pw_dec0_lam-mse0_split-residTrue/",
                            "super/lr8_hr4/var-tasmax/loc-specific-layers_norm-out-normalise_pw_dec0_lam-mse0_split-residTrue/",
                            "super/lr4_hr2/var-tasmax/loc-specific-layers_norm-out-normalise_pw_dec0_lam-mse0_split-residTrue/",
                            "super/lr2_hr1/var-tasmax/loc-specific-layers_norm-out-normalise_pw_dec0_lam-mse0_split-residTrue/",
                        ]
                        burn_ins = [999, 3999, 3999, 1999, 999]
                    
                    elif norm == "subtract_linear" and var == "tasmax":
                        model_dirs = [
                            f"coarse/var-tasmax/hd-100_num-lay-6_norm-out-subtract_linear_norm-in-normalise_scalar_nd-100{alpha_str}_sorted-pred/",
                            f"super/lr16_hr8/var-tasmax/loc-specific-layers_norm-out-subtract_linear_dec0_lam-mse0_split-residTrue{alpha_str}/",
                            f"super/lr8_hr4/var-tasmax/loc-specific-layers_norm-out-subtract_linear_dec0_lam-mse0_split-residTrue{alpha_str}/",
                            f"super/lr4_hr2/var-tasmax/loc-specific-layers_norm-out-subtract_linear_dec0_lam-mse0_split-residTrue{alpha_str}/",
                            f"super/lr2_hr1/var-tasmax/loc-specific-layers_norm-out-subtract_linear_dec0_lam-mse0_split-residTrue{alpha_str}/",
                        ]
                        burn_ins = [999, 3999, 3999, 1999, 999]

                    elif norm == "scale_pw" and var == "pr":
                        model_dirs = [
                            "coarse/var-pr/hd-100_num-lay-6_norm-out-scale_pw_relu_norm-in-normalise_scalar_nd-100_sorted-pred/",
                            "super/lr16_hr8/var-pr/loc-specific-layers_norm-out-scale_pw_relu_dec0_lam-mse0_split-residTrue/",
                            "super/lr8_hr4/var-pr/loc-specific-layers_norm-out-scale_pw_relu_dec0_lam-mse0_split-residTrue/",
                            "super/lr4_hr2/var-pr/loc-specific-layers_norm-out-scale_pw_relu_dec0_lam-mse0_split-residTrue/",
                            "super/lr2_hr1/var-pr/loc-specific-layers_norm-out-scale_pw_relu_dec0_lam-mse0_split-residTrue/",
                        ]
                        burn_ins = [999, 3999, 3999, 1999, 999]

                    elif norm == "subtract_linear" and var == "pr":
                        model_dirs = [
                            f"coarse/var-pr/hd-100_num-lay-6_norm-out-subtract_linear_norm-in-normalise_scalar_nd-100{alpha_str}_sorted-pred/",
                            f"super/lr16_hr8/var-pr/loc-specific-layers_norm-out-subtract_linear_dec0_lam-mse0_split-residTrue{alpha_str}/",
                            f"super/lr8_hr4/var-pr/loc-specific-layers_norm-out-subtract_linear_dec0_lam-mse0_split-residTrue{alpha_str}/",
                            f"super/lr4_hr2/var-pr/loc-specific-layers_norm-out-subtract_linear_dec0_lam-mse0_split-residTrue{alpha_str}/",
                            f"super/lr2_hr1/var-pr/loc-specific-layers_norm-out-subtract_linear_dec0_lam-mse0_split-residTrue{alpha_str}/",
                        ]
                        burn_ins = [499, 3999, 3999, 1999, 999]

                    # Prefix logic
                    
                    # Check each of the 5 resolution levels
                    prefix_with = f"results/{exp}/{dom}/with-orog/"                    
                    prefix_none = f"results/{exp}/{dom}/no-orog/"
                    root = "/r/scratch/users/mschillinger/code/cordexbench/enscale"

                    for i in range(5):
                        total_count += 1
                        
                        # NEW OROG LOGIC: 
                        # Use 'no-orog' if it's the coarse model (i==0) OR if orography is disabled.
                        # Otherwise, use 'with-orog'.
                        save_dir_prefix = prefix_none if (i == 0 or not orog) else prefix_with
                        
                        ckpt_path = os.path.join(root, save_dir_prefix, model_dirs[i], f"model_{burn_ins[i]}.pt")
                        path_no_root = os.path.join(save_dir_prefix, model_dirs[i], f"model_{burn_ins[i]}.pt")
                        
                        if os.path.exists(ckpt_path):
                            found_count += 1
                        else:
                            print(f"{'MISSING':<10} | {path_no_root}")
                            missing_models.append(path_no_root)

    print("-" * 80)
    print(f"Summary: Found {found_count}/{total_count} models.")
    if not missing_models:
        print("All models are trained and ready!")
    else:
        print(f"Missing {len(missing_models)} checkpoints.")

if __name__ == "__main__":
    check_model_availability()

STATUS     | PATH
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Summary: Found 180/180 models.
All models are trained and ready!


In [2]:
import os

def check_samples_existence():
    # Base path provided
    root_samples = "/r/scratch/groups/nm/downscaling/samples_cordexbench/"
    
    # Options from your requirements
    experiments = ["Emulator_hist_future", "ESD_pseudo_reality"]
    domains = ["ALPS", "NZ", "SA"]
    target_vars = ["tasmax", "pr"] # Following your previous patterns
    
    print(f"{'EXPERIMENT':<22} | {'DOMAIN':<6} | {'VAR':<8} | {'STATUS'}")
    print("-" * 60)

    found_any = False
    for exp in experiments:
        for dom in domains:
            for var in target_vars:
                # Construct path: root / exp / dom / "no-orog" / var / "linear_pred"
                target_path = os.path.join(root_samples, exp, dom, "no-orog", var, "linear_pred")
                
                if os.path.exists(target_path) and len(os.listdir(target_path)) == 12:  # Check if directory exists and is not empty
                    print(f"{exp[:20]:<22} | {dom:<6} | {var:<8} | COMPLETE")
                    found_any = True
                else:
                    # Optional: uncomment if you want to see what is missing
                    print(f"{exp[:20]:<22} | {dom:<6} | {var:<8} | MISSING")
                    pass
    
    if not found_any:
        print("No matching sample directories found. Check if the root path is mounted.")

if __name__ == "__main__":
    check_samples_existence()

EXPERIMENT             | DOMAIN | VAR      | STATUS
------------------------------------------------------------
Emulator_hist_future   | ALPS   | tasmax   | MISSING
Emulator_hist_future   | ALPS   | pr       | COMPLETE
Emulator_hist_future   | NZ     | tasmax   | COMPLETE
Emulator_hist_future   | NZ     | pr       | COMPLETE
Emulator_hist_future   | SA     | tasmax   | COMPLETE
Emulator_hist_future   | SA     | pr       | COMPLETE
ESD_pseudo_reality     | ALPS   | tasmax   | COMPLETE
ESD_pseudo_reality     | ALPS   | pr       | COMPLETE
ESD_pseudo_reality     | NZ     | tasmax   | COMPLETE
ESD_pseudo_reality     | NZ     | pr       | COMPLETE
ESD_pseudo_reality     | SA     | tasmax   | COMPLETE
ESD_pseudo_reality     | SA     | pr       | COMPLETE


In [4]:
import os

def check_samples_existence():
    # Base path provided
    root_samples = "/r/scratch/groups/nm/downscaling/samples_cordexbench/"
    
    # Options from your requirements
    experiments = ["Emulator_hist_future", "ESD_pseudo_reality"]
    domains = ["ALPS", "NZ", "SA"]
    target_vars = ["tasmax", "pr"] # Following your previous patterns
    
    print(f"{'EXPERIMENT':<22} | {'DOMAIN':<6} | {'VAR':<8} | {'STATUS'}")
    print("-" * 60)

    found_any = False
    for exp in experiments:
        for dom in domains:
            for var in target_vars:
                # Construct path: root / exp / dom / "no-orog" / var / "linear_pred"
                
                target_path = os.path.join(root_samples, exp, dom, "with-orog", var, "maybritt_scalepw_v2")
                
                if os.path.exists(target_path) and len(os.listdir(target_path)) > 11:  # Check if directory exists and is not empty
                    print(f"{exp[:20]:<22} | {dom:<6} | {var:<8} | COMPLETE")
                    found_any = True
                else:
                    # Optional: uncomment if you want to see what is missing
                    print(f"{exp[:20]:<22} | {dom:<6} | {var:<8} | MISSING")
                    pass
    
    if not found_any:
        print("No matching sample directories found. Check if the root path is mounted.")

if __name__ == "__main__":
    check_samples_existence()

EXPERIMENT             | DOMAIN | VAR      | STATUS
------------------------------------------------------------
Emulator_hist_future   | ALPS   | tasmax   | MISSING
Emulator_hist_future   | ALPS   | pr       | COMPLETE
Emulator_hist_future   | NZ     | tasmax   | MISSING
Emulator_hist_future   | NZ     | pr       | COMPLETE
Emulator_hist_future   | SA     | tasmax   | MISSING
Emulator_hist_future   | SA     | pr       | COMPLETE
ESD_pseudo_reality     | ALPS   | tasmax   | MISSING
ESD_pseudo_reality     | ALPS   | pr       | COMPLETE
ESD_pseudo_reality     | NZ     | tasmax   | MISSING
ESD_pseudo_reality     | NZ     | pr       | COMPLETE
ESD_pseudo_reality     | SA     | tasmax   | MISSING
ESD_pseudo_reality     | SA     | pr       | COMPLETE


In [4]:
import os

def check_samples_existence():
    # Base path provided
    root_samples = "/r/scratch/groups/nm/downscaling/samples_cordexbench/"
    
    # Options from your requirements
    experiments = ["Emulator_hist_future", "ESD_pseudo_reality"]
    domains = ["ALPS", "NZ", "SA"]
    target_vars = ["tasmax", "pr"] # Following your previous patterns
    
    print(f"{'EXPERIMENT':<22} | {'DOMAIN':<6} | {'VAR':<8} | {'STATUS'}")
    print("-" * 60)

    found_any = False
    for exp in experiments:
        for dom in domains:
            for var in target_vars:
                # Construct path: root / exp / dom / "no-orog" / var / "linear_pred"
                
                target_path = os.path.join(root_samples, exp, dom, "no-orog", var, "maybritt_normpw_v2")
                
                if os.path.exists(target_path):
                    ckpt_file = os.path.join(target_path, "ckpt_dirs.txt")
                    if os.path.exists(ckpt_file):
                        with open(ckpt_file, "r") as f:
                            first_entry = f.readline().strip()
                            print(first_entry)
                    print(f"{exp[:20]:<22} | {dom:<6} | {var:<8} | COMPLETE")
                    found_any = True
                else:
                    # Optional: uncomment if you want to see what is missing
                    print(f"{exp[:20]:<22} | {dom:<6} | {var:<8} | MISSING")
                    pass
    
    if not found_any:
        print("No matching sample directories found. Check if the root path is mounted.")

if __name__ == "__main__":
    check_samples_existence()

EXPERIMENT             | DOMAIN | VAR      | STATUS
------------------------------------------------------------
Model 0 checkpoint: coarse/var-tasmax/hd-100_num-lay-6_norm-out-normalise_pw_norm-in-normalise_scalar_nd-100_sorted-pred/model_999.pt
Emulator_hist_future   | ALPS   | tasmax   | COMPLETE
Emulator_hist_future   | ALPS   | pr       | MISSING
Model 0 checkpoint: coarse/var-tasmax/hd-100_num-lay-6_norm-out-normalise_pw_norm-in-normalise_scalar_nd-100_sorted-pred/model_999.pt
Emulator_hist_future   | NZ     | tasmax   | COMPLETE
Emulator_hist_future   | NZ     | pr       | MISSING
Model 0 checkpoint: coarse/var-tasmax/hd-100_num-lay-6_norm-out-normalise_pw_norm-in-normalise_scalar_nd-100_sorted-pred/model_999.pt
Emulator_hist_future   | SA     | tasmax   | COMPLETE
Emulator_hist_future   | SA     | pr       | MISSING
Model 0 checkpoint: coarse/var-tasmax/hd-100_num-lay-6_norm-out-normalise_pw_norm-in-normalise_scalar_nd-100_sorted-pred/model_999.pt
ESD_pseudo_reality     | ALPS 

In [6]:
import os

def check_samples_existence():
    # Base path provided
    root_samples = "/r/scratch/groups/nm/downscaling/samples_cordexbench/"
    
    # Options from your requirements
    experiments = ["Emulator_hist_future", "ESD_pseudo_reality"]
    domains = ["ALPS", "NZ", "SA"]
    target_vars = ["tasmax", "pr"] # Following your previous patterns
    
    print(f"{'EXPERIMENT':<22} | {'DOMAIN':<6} | {'VAR':<8} | {'STATUS'}")
    print("-" * 60)

    found_any = False
    for exp in experiments:
        for dom in domains:
            for var in target_vars:
                # Construct path: root / exp / dom / "no-orog" / var / "linear_pred"
                
                target_path = os.path.join(root_samples, exp, dom, "no-orog", var, "maybritt_linex_v1")
                
                if os.path.exists(target_path) and len(os.listdir(target_path)) > 11:  # Check if directory exists and is not empty
                    print(f"{exp[:20]:<22} | {dom:<6} | {var:<8} | COMPLETE")
                    found_any = True
                else:
                    # Optional: uncomment if you want to see what is missing
                    print(f"{exp[:20]:<22} | {dom:<6} | {var:<8} | MISSING")
                    pass
    
    if not found_any:
        print("No matching sample directories found. Check if the root path is mounted.")

if __name__ == "__main__":
    check_samples_existence()

EXPERIMENT             | DOMAIN | VAR      | STATUS
------------------------------------------------------------
Emulator_hist_future   | ALPS   | tasmax   | COMPLETE
Emulator_hist_future   | ALPS   | pr       | COMPLETE
Emulator_hist_future   | NZ     | tasmax   | COMPLETE
Emulator_hist_future   | NZ     | pr       | COMPLETE
Emulator_hist_future   | SA     | tasmax   | COMPLETE
Emulator_hist_future   | SA     | pr       | COMPLETE
ESD_pseudo_reality     | ALPS   | tasmax   | MISSING
ESD_pseudo_reality     | ALPS   | pr       | MISSING
ESD_pseudo_reality     | NZ     | tasmax   | MISSING
ESD_pseudo_reality     | NZ     | pr       | MISSING
ESD_pseudo_reality     | SA     | tasmax   | MISSING
ESD_pseudo_reality     | SA     | pr       | MISSING
